In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

## Data reading

In [2]:
df=pd.read_csv("resources/Bengaluru_House_Data.csv")
df.head()    # Data read successfully

,area_type,availability,location,size,society,total_sqft,bath,balcony,price(in Rs. Lakhs)
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


## Data cleaning

In [3]:
df.shape    # rows, columns

(13320, 9)

In [4]:
# Get the count of different area types
df.groupby("area_type")["area_type"].agg("count")

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

In [8]:
# Let us assume that area_type, availability, society are not important factors that affect price
# So, we can drop these columns
df1=df.drop(["area_type", "availability", "society", "balcony"], axis="columns")
df1.head()

,location,size,total_sqft,bath,price(in Rs. Lakhs)
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [9]:
# Get the no. of rows per column where value is NULL
df1.isnull().sum()

location                1
size                   16
total_sqft              0
bath                   73
price(in Rs. Lakhs)     0
dtype: int64

In [10]:
# drop the rows where any column is NULL
df2=df1.dropna()
df2.isnull().sum()

location               0
size                   0
total_sqft             0
bath                   0
price(in Rs. Lakhs)    0
dtype: int64

In [12]:
df2.shape

(13246, 5)

In [13]:
# Get all the unique values in the column "size"
df2["size"].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [15]:
# Create a new column "BHK" that will store the values from the "size" column
df2["BHK"]=df2["size"].apply(lambda x: int(x.split(' ')[0]))
df2.head()

C:\Users\Rishit\AppData\Local\Temp\ipykernel_3948\3491002690.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["BHK"]=df2["size"].apply(lambda x: int(x.split(' ')[0]))


,location,size,total_sqft,bath,price(in Rs. Lakhs),BHK
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [16]:
# Get all the unique values in the newly created column
df2["BHK"].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [18]:
# Get all the houses with more than 20 bedrooms
df2[df2["BHK"]>20]

,location,size,total_sqft,bath,price(in Rs. Lakhs),BHK
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [24]:
# Get all the unique total_sqft
df2["total_sqft"].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [25]:
# Define a function that will return whether a given value can be converted to float or not(is it a range etc.)
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [30]:
# Get all the rows where the "total_sqft" is not convertible to a float(it is a range or something else)
df2[~df2["total_sqft"].apply(is_float)].head(20)

,location,size,total_sqft,bath,price(in Rs. Lakhs),BHK
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4
